In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.inspection import permutation_importance
from fairlearn.metrics import MetricFrame
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, demographic_parity_ratio 
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_sample_weight


In [2]:
df = pd.read_csv('car_insurance_claim.csv')

In [3]:
df.describe()

,ID,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE,CLAIM_FLAG
count,1.030200e+04,10302.000000,10295.000000,10302.000000,9754.000000,10302.000000,10302.000000,10302.000000,10302.000000,9663.000000,10302.000000
mean,4.956631e+08,0.169288,44.837397,0.720443,10.474062,33.416424,5.329159,0.800718,1.710153,8.298148,0.266550
std,2.864675e+08,0.506512,8.606445,1.116323,4.108943,15.869687,4.110795,1.154079,2.159015,5.714450,0.442177
min,6.317500e+04,0.000000,16.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,-3.000000,0.000000
25%,2.442869e+08,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.970043e+08,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000,0.000000
75%,7.394551e+08,0.000000,51.000000,1.000000,13.000000,44.000000,7.000000,2.000000,3.000000,12.000000,1.000000
max,9.999264e+08,4.000000,81.000000,5.000000,23.000000,142.000000,25.000000,5.000000,13.000000,28.000000,1.000000


In [4]:
df.head(2)

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,...,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,...,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban


In [5]:
a = sum(df['CLAIM_FLAG'] == False)
print(a)

7556


In [6]:
b = sum(df['CLAIM_FLAG'] == True)
print(b)

2746


In [7]:
df = df.drop(columns=['ID','BIRTH'],axis=1)

In [8]:
numerical = [
    'KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME',
    'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM',
    'CLM_FREQ', 'MVR_PTS', 'CLM_AMT', 'CAR_AGE'
]

categorical = [
  'PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION',
    'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY'
]

df[categorical] = df[categorical].apply(lambda x: x.fillna(x.mode()[0]))

        
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$','').replace(',',''))
    return x

for col in ['INCOME', 'HOME_VAL', 'BLUEBOOK', 'OLDCLAIM', 'CLM_AMT']:
    df[col] = df[col].apply(clean_currency)


print("Numerical columns:", numerical)
print("Categorical columns:", categorical)

Numerical columns: ['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'HOME_VAL', 'TRAVTIME', 'BLUEBOOK', 'TIF', 'OLDCLAIM', 'CLM_FREQ', 'MVR_PTS', 'CLM_AMT', 'CAR_AGE']
Categorical columns: ['PARENT1', 'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_USE', 'CAR_TYPE', 'RED_CAR', 'REVOKED', 'URBANICITY']


In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical)
    ]
)

In [10]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(random_state=42))
])

In [11]:
X = df.drop('CLAIM_FLAG', axis=1)
y = df['CLAIM_FLAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [12]:
print(df.columns)

Index(['KIDSDRIV', 'AGE', 'HOMEKIDS', 'YOJ', 'INCOME', 'PARENT1', 'HOME_VAL',
       'MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'TRAVTIME', 'CAR_USE',
       'BLUEBOOK', 'TIF', 'CAR_TYPE', 'RED_CAR', 'OLDCLAIM', 'CLM_FREQ',
       'REVOKED', 'MVR_PTS', 'CLM_AMT', 'CAR_AGE', 'CLAIM_FLAG', 'URBANICITY'],
      dtype='object')


In [13]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1512
           1       1.00      0.97      0.98       549

    accuracy                           0.99      2061
   macro avg       0.99      0.98      0.99      2061
weighted avg       0.99      0.99      0.99      2061

ROC-AUC Score: 0.9995940189473888


In [15]:
groups = ['GENDER', 'EDUCATION', 'MSTATUS', 'PARENT1', 'OCCUPATION', 'URBANICITY']
results = []

In [16]:

for group in groups:
    for value in df[group].unique():
        mask = X_test[group] == value
        if mask.sum() > 0:
            group_y_test = y_test[mask]
            group_y_pred = y_pred[mask]
            group_y_pred_proba = y_pred_proba[mask]
            try:
                roc_auc = roc_auc_score(group_y_test, group_y_pred_proba)
            except ValueError:
                roc_auc = float('nan')
            tn, fp, fn, tp = confusion_matrix(group_y_test, group_y_pred).ravel()
            results.append({
                "Group": group,
                "Value": value,
                "Accuracy": (group_y_test == group_y_pred).mean(),
                "ROC-AUC": roc_auc,
                "TP": tp,
                "TN": tn,
                "FP": fp,
                "FN": fn
            })
            for a in results:
                print(a)


{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.9915878023133544, 'ROC-AUC': 0.9993340922026182, 'TP': 243, 'TN': 700, 'FP': 0, 'FN': 8}
{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.9915878023133544, 'ROC-AUC': 0.9993340922026182, 'TP': 243, 'TN': 700, 'FP': 0, 'FN': 8}
{'Group': 'GENDER', 'Value': 'z_F', 'Accuracy': 0.990990990990991, 'ROC-AUC': 0.9999049492511655, 'TP': 288, 'TN': 812, 'FP': 0, 'FN': 10}
{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.9915878023133544, 'ROC-AUC': 0.9993340922026182, 'TP': 243, 'TN': 700, 'FP': 0, 'FN': 8}
{'Group': 'GENDER', 'Value': 'z_F', 'Accuracy': 0.990990990990991, 'ROC-AUC': 0.9999049492511655, 'TP': 288, 'TN': 812, 'FP': 0, 'FN': 10}
{'Group': 'EDUCATION', 'Value': 'PhD', 'Accuracy': 0.9898477157360406, 'ROC-AUC': 0.9985564778058462, 'TP': 32, 'TN': 163, 'FP': 0, 'FN': 2}
{'Group': 'GENDER', 'Value': 'M', 'Accuracy': 0.9915878023133544, 'ROC-AUC': 0.9993340922026182, 'TP': 243, 'TN': 700, 'FP': 0, 'FN': 8}
{'Group': 'GENDER', 'Value': 'z_F

In [30]:
def evaluate_fairness(y_true, y_pred, sensitive_features, group_name):
    eod = equalized_odds_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    
    dpd = demographic_parity_difference(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    di_ratio = demographic_parity_ratio(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    print(f'\n group is {group_name}')
    print(f"Demographic Parity Ratio: {di_ratio:.4f}")
    print(f"Equalized Odds Difference: {eod:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    
    
    positive_rates = {}
    for group_value in sensitive_features.unique():
        mask = sensitive_features == group_value
        group_y_pred = y_pred[mask]
        positive_rate = group_y_pred.mean()
        positive_rates[group_value] = positive_rate
        print(f"Subgroup: {group_value}, Positive Prediction Rate: {positive_rate:.4f}")
    
    # Identify privileged and unprivileged groups
    max_rate = max(positive_rates.values())
    min_rate = min(positive_rates.values())
    
    for group_value, rate in positive_rates.items():
        if rate == min_rate:
            print(f"--> Privileged Group: {group_value} (Positive Rate: {rate:.4f})")
        elif rate == max_rate:
            print(f"--> Unprivileged Group: {group_value} (Positive Rate: {rate:.4f})")

In [31]:
for group in groups:
    
    evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is GENDER
Demographic Parity Ratio: 0.9848
Equalized Odds Difference: 0.0017
Demographic Parity Difference: 0.0039
Subgroup: M, Positive Prediction Rate: 0.2555
Subgroup: z_F, Positive Prediction Rate: 0.2595
--> Privileged Group: M (Positive Rate: 0.2555)
--> Unprivileged Group: z_F (Positive Rate: 0.2595)

 group is EDUCATION
Demographic Parity Ratio: 0.4776
Equalized Odds Difference: 0.0487
Demographic Parity Difference: 0.1777
Subgroup: z_High School, Positive Prediction Rate: 0.3401
Subgroup: Bachelors, Positive Prediction Rate: 0.2249
Subgroup: Masters, Positive Prediction Rate: 0.1856
Subgroup: <High School, Positive Prediction Rate: 0.3192
Subgroup: PhD, Positive Prediction Rate: 0.1624
--> Unprivileged Group: z_High School (Positive Rate: 0.3401)
--> Privileged Group: PhD (Positive Rate: 0.1624)

 group is MSTATUS
Demographic Parity Ratio: 0.6720
Equalized Odds Difference: 0.0006
Demographic Parity Difference: 0.1053
Subgroup: z_No, Positive Prediction Rate: 0.3211
Sub

In [19]:
display(df.head(2))

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,0,60.0,0,11.0,67349.0,No,0.0,z_No,M,PhD,...,Minivan,yes,4461.0,2,No,3,0.0,18.0,0,Highly Urban/ Urban
1,0,43.0,0,11.0,91449.0,No,257252.0,z_No,M,z_High School,...,Minivan,yes,0.0,0,No,0,0.0,1.0,0,Highly Urban/ Urban


We see that the columns occupation, mstatus, parent1, and occupation indicate the presence of bias in our dataset.

In [20]:
from aif360.metrics import BinaryLabelDatasetMetric, DatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
metric_orig_train = BinaryLabelDatasetMetric(X_train['EDUCATION'], 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Original training dataset")
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migratin

NameError: name 'unprivileged_groups' is not defined

In [ ]:
dataset_orig = df(
    protected_attribute_names=['EDUCATION'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x = 25]      # age >=25 is considered privileged # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]